<a href="https://colab.research.google.com/github/theostoican/Guided-Research/blob/main/gr_dino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoFeatureExtractor, ViTMAEForPreTraining
from transformers import ViTModel

from PIL import Image
import requests
import os
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt


url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)
image = Image.open('ball1.jpg')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.3 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('facebook/dino-vitb16')
model = ViTModel.from_pretrained('facebook/dino-vitb16', output_attentions=True)

# feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/vit-mae-base', output_attentions=True)
# model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')

inputs = feature_extractor(images=image, return_tensors="pt")
print(inputs.pixel_values.shape)

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 3, 224, 224])


In [ ]:
outputs = model(**inputs)

In [ ]:
outputs = model(inputs.pixel_values)

In [ ]:
print(len(outputs.attentions))

12


In [ ]:
print(outputs.attentions[-1].shape)

torch.Size([1, 12, 197, 197])


In [ ]:
print(outputs.attentions[-1][0][7][0])
print(outputs.attentions[-1][0][7][100])
print(outputs.attentions[-1][0][7][196])

tensor([3.8301e-01, 2.5268e-03, 4.9462e-03, 2.3344e-03, 1.5286e-03, 1.2671e-03,
        1.3134e-03, 1.0374e-03, 3.9219e-03, 7.5686e-03, 6.9646e-04, 8.1386e-04,
        7.8513e-04, 6.5994e-04, 9.1999e-04, 1.2134e-03, 3.4103e-03, 7.9474e-04,
        3.5355e-04, 4.3332e-04, 1.6920e-04, 4.6929e-04, 2.7335e-03, 1.3403e-03,
        1.8870e-04, 3.6438e-04, 3.1944e-04, 4.3322e-04, 7.5048e-04, 3.8758e-03,
        3.1147e-03, 2.1784e-03, 2.9263e-03, 6.2633e-04, 3.1049e-03, 8.2182e-03,
        1.0265e-02, 1.0824e-02, 4.9786e-03, 1.4225e-03, 2.8969e-04, 1.7273e-03,
        2.9927e-03, 1.4243e-03, 1.0561e-03, 1.0598e-03, 7.5785e-04, 6.3571e-03,
        1.8822e-02, 1.3318e-02, 3.2428e-02, 2.1203e-02, 2.1195e-02, 3.5006e-02,
        1.8567e-03, 2.1059e-03, 2.0263e-03, 8.6566e-04, 9.3117e-04, 9.6254e-04,
        1.9606e-03, 6.0618e-03, 4.6846e-03, 5.7893e-03, 8.8193e-03, 9.9019e-03,
        9.5359e-03, 8.6814e-03, 3.4023e-03, 6.3596e-04, 1.0063e-03, 5.4717e-03,
        6.9864e-03, 7.9783e-03, 3.3660e-

In [ ]:
attentions = outputs.attentions[-1].detach()

nh = attentions.shape[1] # number of heads


patch_size = 16
print (inputs.pixel_values.shape)
w, h = inputs.pixel_values.shape[-2] - inputs.pixel_values.shape[-2] % patch_size, inputs.pixel_values.shape[-1] - inputs.pixel_values.shape[-1] % patch_size
inputs.pixel_values = inputs.pixel_values[:, :w, :h].unsqueeze(0)

w_featmap = inputs.pixel_values.shape[-2] // patch_size
h_featmap = inputs.pixel_values.shape[-1] // patch_size

attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

attentions = attentions.reshape(nh, w_featmap, h_featmap)
attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0].cpu().numpy()

# save attentions heatmaps
os.makedirs('maps', exist_ok=True)
torchvision.utils.save_image(torchvision.utils.make_grid(inputs.pixel_values, normalize=True, scale_each=True),
                             os.path.join('maps', "img.png"))
for j in range(nh):
    fname = os.path.join('maps', "attn-head" + str(j) + ".png")
    print(attentions[j].shape)
    break
    plt.imsave(fname=fname, arr=attentions[j], format='png')
    print(f"{fname} saved.")

torch.Size([1, 3, 224, 224])
(224, 224)


In [ ]:
print(outputs.keys())

odict_keys(['last_hidden_state', 'pooler_output', 'attentions'])


In [ ]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 197, 768])
